In [ ]:
import re
import collections
import math
import xlrd
import xlsxwriter
import numpy as np
import datetime
import codecs
from scipy.sparse import csr_matrix
from scipy.sparse import csc_matrix
from scipy.spatial.distance import cdist
from scipy.spatial.distance import cosine

In [ ]:
def add_months(s, p):
    ''' 在日期s上加上p个月，如果日期不存在，顺延到下个月1号 '''
    p_years = s.year + p / 12
    p_months = s.month + p % 12 
    if p_months > 12:
        p_years += 1
        p_months = p_months % 12
    
    try:
        rs = datetime.datetime(p_years, p_months, s.day)
    except ValueError, e:
        if p_months == 12:
            rs = datetime.datetime(p_years + 1, 1, 1)
        else:
            rs = datetime.datetime(p_years, p_months+1, 1)
    return rs

def next_quater(cur_quater):
    return add_months(cur_quater, 3)

def next_half(cur_half):
    return add_months(cur_half, 6)

class SharesInfo(object):
    def __init__(self, shares, marketValue, proportion):
        self.shares_ = shares
        self.marketValue_ = marketValue
        self.proportion_ = proportion
    
    def __str__(self):
        return "%f %f %f"%(self.shares_, self.marketValue_, self.proportion_)
    def __repr__(self):
        return "%f %f %f"%(self.shares_, self.marketValue_, self.proportion_)


half_date = [datetime.datetime(2003,1,1), datetime.datetime(2003,7,1)] \
        + [datetime.datetime(v1,v2,1) for v1 in xrange(2004, 2018) for v2 in (1,7)]

In [ ]:
g_raw_data = collections.defaultdict(lambda : collections.defaultdict(dict))
g_record = 0
g_stock_set = set()
def load_data(file_name="Fund_Portfolio_Stock2003-2007.txt"):
    global g_record
    with codecs.open(file_name, "r", encoding="utf_16") as ifid:
        line_num = 0
        for line in ifid:
            if line_num > 0 and len(line) > 5:
                try:
                    MasterFundCode, ReportTypeID, Startdate, EndDate, Rank, Symbol, StockName, Shares, MarketValue, Proportion \
                        = line.strip().split('\t')
                    Startdate = datetime.datetime.strptime(Startdate, "%Y-%m-%d")
                    EndDate = datetime.datetime.strptime(EndDate, "%Y-%m-%d")
                    if int(ReportTypeID) in (5,6):
                        real_start_date = datetime.datetime(Startdate.year, 6*int(ReportTypeID)-29, 1)
                        real_idx = half_date.index(real_start_date)
                        
                        if Startdate <= EndDate:
                            g_raw_data[MasterFundCode][Symbol][real_start_date] = SharesInfo(float(Shares), float(MarketValue), float(Proportion))
                            g_stock_set.add(Symbol)
                            g_record += 1
                        else:
                            print u"error at %s linenum %d:%s %s %s"  %(file_name, line_num+1, MasterFundCode, Symbol, Startdate)
                except ValueError, e:
                    print line.strip(), file_name, line_num+1
            line_num += 1


                            
load_data("Fund_Portfolio_Stock2003-2007.txt")
load_data("Fund_Portfolio_Stock2008-2012.txt")
load_data("Fund_Portfolio_Stock2013-2017.txt")
print "加载%d只基金%d条数据，共%d只股票" %(len(g_raw_data), g_record, len(g_stock_set))


In [ ]:
print g_raw_data[u'040001'][u'600028']
print g_holding_num_data[u'040001'][datetime.datetime(2004, 1, 1, 0, 0)].sum()

In [ ]:
# 加载股票半年收益率
g_change_rate = collections.defaultdict(lambda : np.zeros(len(g_stock_set)))
g_stock_map = {sym_: i_ for i_, sym_ in enumerate(sorted(g_stock_set))}
with xlrd.open_workbook("stock_list.xlsx") as wb:
    ws = wb.sheet_by_index(0)
    for i in xrange(2, ws.nrows):
        stock_sym = ws.cell(i,0).value
        cur_half = datetime.datetime(2003,1,1)
        for j in xrange(1, ws.ncols):
            g_change_rate[cur_half][g_stock_map[stock_sym]] = ws.cell(i,j).value/100
            cur_half = next_half(cur_half)
                

In [ ]:
#dump g_stock_map
with xlsxwriter.Workbook("stock_list.xlsx") as wb:
    ws = wb.add_worksheet()
    wt_idx = 0
    for stock_name in sorted(g_stock_set):
        ws.write(wt_idx, 0, stock_name)
        wt_idx += 1

In [ ]:
# 加载基金的半年持仓情况
g_holding_num_data = collections.defaultdict(lambda : collections.defaultdict(lambda : np.zeros(len(g_stock_set))))
g_stock_map = {sym_: i_ for i_, sym_ in enumerate(sorted(g_stock_set))}
for fund_ in g_raw_data:
    for sym_ in g_raw_data[fund_]:
        for cur_ in g_raw_data[fund_][sym_]:
            g_holding_num_data[fund_][cur_][g_stock_map[sym_]] = g_raw_data[fund_][sym_][cur_].proportion_ / 100
# for fund_ in g_holding_num_data:
#     for cur_ in g_holding_num_data[fund_]:
#         if g_holding_num_data[fund_][cur_].sum() != 1:
#             print "%s %s"%(fund_, cur_)

In [ ]:
print g_change_rate[datetime.datetime(2004, 1, 1, 0, 0)]
print g_holding_num_data[u'000001'][datetime.datetime(2003, 7, 1, 0, 0)].sum()
print g_holding_num_data[u'000001'][datetime.datetime(2004, 1, 1, 0, 0)]

In [ ]:
g_trade_vector = collections.defaultdict(dict)
for fund_ in g_holding_num_data:
    start_date_ = sorted(g_holding_num_data[fund_].keys())[0]
    end_date_ = sorted(g_holding_num_data[fund_].keys())[-1]
    for i in xrange(half_date.index(start_date_)+1, half_date.index(end_date_)+1):
        if half_date[i] in g_holding_num_data[fund_] and half_date[i] in g_change_rate:
            if half_date[i-1] in g_holding_num_data[fund_]:
                fixed_100 = np.inner(g_change_rate[half_date[i]], g_holding_num_data[fund_][half_date[i-1]])
                if fixed_100 == 0:
                    print "%s %s %s %f %f"%(fund_, half_date[i], half_date[i-1], \
                                            g_change_rate[half_date[i]].sum(), g_holding_num_data[fund_][half_date[i-1]].sum())
                    continue
                fixed_T_1 = g_holding_num_data[fund_][half_date[i-1]] / fixed_100
                g_trade_vector[half_date[i]][fund_] = g_holding_num_data[fund_][half_date[i]] - fixed_T_1
            else:
                g_trade_vector[half_date[i]][fund_] = g_holding_num_data[fund_][half_date[i]]
        else:
            if half_date[i] not in g_holding_num_data[fund_]:
                print u"error at %s %s"%(fund_, half_date[i])
            else:
                print u"error at change rate %s"%(half_date[i])

In [ ]:
print g_trade_vector[datetime.datetime(2004,1,1)]

In [ ]:
g_rank_raw_data = collections.defaultdict(dict)
g_fund_set = set()
with xlrd.open_workbook(u'half_pref_order_result.xlsx') as wb:
    ws = wb.sheet_by_index(0)
    for r in xrange(1, ws.nrows):
        fund_id = ws.cell(r, 0).value
        half = datetime.datetime(int(ws.cell(r,1).value[:4]), int(ws.cell(r,1).value[5])*6-5, 1)
        order = float(ws.cell(r,2).value)
        
        if half in half_date:
            g_rank_raw_data[half][fund_id] = order
            g_fund_set.add(fund_id)
        else:
            print "error at %s %s %f"%(fund_id, half, order)
            
g_sorted_fund = sorted(g_fund_set)
g_fund_map = {fund: i for i, fund in enumerate(g_sorted_fund)}
g_rank = collections.defaultdict(lambda : np.empty(len(g_sorted_fund), dtype='int64'))
g_funds_num = collections.defaultdict(int)
g_rank_value = {}
tmp_order = collections.defaultdict(lambda : np.full(len(g_sorted_fund), -1.0))

for half in g_rank_raw_data:
    for fund in g_rank_raw_data[half]:
        tmp_order[half][g_fund_map[fund]] = g_rank_raw_data[half][fund]
        g_funds_num[half] += 1
        
for half in tmp_order:
    sorted_idx = np.argsort(tmp_order[half])[::-1]
    g_rank_value[half] = np.sort(tmp_order[half])[::-1]
    g_rank[half][sorted_idx] = np.arange(len(g_sorted_fund))

In [ ]:
debug_funds = u'000001'
debug_half = datetime.datetime(2003,7,1)
print g_rank_value[debug_half]
print g_rank[debug_half]

In [ ]:
def calc_sim1(trade_vec):
    leading_sim_ = collections.defaultdict(dict)
    current_sim_ = collections.defaultdict(dict)
    following_sim_ = collections.defaultdict(dict)
    following_advance_10_sim_ = collections.defaultdict(dict)
    
    start_idx_ = half_date.index(datetime.datetime(2004,1,1))
    end_idx_ = half_date.index(datetime.datetime(2017,7,1))
    
    for i in xrange(start_idx_, end_idx_):
        current_date = half_date[i]
        # 去除变化为0的funds，并且排序
        sorted_current_funds = sorted([funds_ for funds_ in trade_vec[current_date] \
                                       if trade_vec[current_date][funds_].sum()!=0])   
        
        for fund_ in sorted_current_funds:
            peer = np.array([trade_vec[current_date][other_fund] \
                             for other_fund in sorted_current_funds if other_fund != fund_]).mean(0)      
            current_sim_[fund_][current_date] = (cosine(trade_vec[current_date][fund_], peer), len(sorted_current_funds)-1)
    
    
    cross_start_idx = half_date.index(datetime.datetime(2004,1,1))
    cross_end_idx = half_date.index(datetime.datetime(2017,7,1))
    for i in xrange(cross_start_idx, cross_end_idx):
        previous_date = half_date[i-1]
        current_date= half_date[i]
        
        sorted_previous_funds = sorted([funds_ for funds_ in trade_vec[previous_date] \
                                        if trade_vec[previous_date][funds_].sum()!=0])
        sorted_current_funds = sorted([funds_ for funds_ in trade_vec[current_date] \
                                       if trade_vec[current_date][funds_].sum()!=0])
        
        for fund_ in sorted_previous_funds:
            current_pear = np.array([trade_vec[current_date][other_fund] \
                                    for other_fund in sorted_current_funds if other_fund != fund_]).mean(0)
            peer_num = len(sorted_current_funds) - 1 if fund_ in sorted_current_funds else len(sorted_current_funds)
            leading_sim_[fund_][previous_date] = (cosine(trade_vec[previous_date][fund_], current_pear), peer_num)
                    
        for fund_ in sorted_current_funds:
            previous_peer =  np.array([trade_vec[previous_date][other_fund] \
                                      for other_fund in sorted_previous_funds if other_fund != fund_]).mean(0)
            peer_num = len(sorted_previous_funds) - 1 if fund_ in sorted_previous_funds else len(sorted_previous_funds)
            following_sim_[fund_][current_date] = (cosine(trade_vec[current_date][fund_], previous_peer), peer_num)
            
        for i_, fund_ in enumerate(sorted_current_funds):
            my_order_ = g_rank_raw_data[previous_date].get(fund_, -1.0)
            top10_num_ = int(math.ceil(g_funds_num[previous_date] * 0.1))
            top10_order_ = g_rank_value[previous_date][top10_num_]
            order_ = top10_order_ if top10_order_ > my_order_ else my_order_
            
            sorted_previous_advance_flag = [True if g_rank_raw_data[previous_date].get(foo_fund, -2.0) > order_ else False \
                                         for foo_fund in sorted_previous_funds]
            if any(sorted_previous_advance_flag):
                previous_peer =  np.array([trade_vec[previous_date][other_fund] \
                                      for other_fund in sorted_previous_funds \
                                       if g_rank_raw_data[previous_date].get(other_fund, -2.0) > order_]).mean(0)
            
                following_advance_10_sim_[fund_][current_date] = (cosine(trade_vec[current_date][fund_], previous_peer), \
                                                              sum(sorted_previous_advance_flag))

                
    return leading_sim_, current_sim_, following_sim_, following_advance_10_sim_

g_leading_sim, g_current_sim, g_following_sim, g_following_advance_10_sim = calc_sim1(g_trade_vector)

In [ ]:
g_jingli_leading_sim = collections.defaultdict(lambda :collections.defaultdict(dict))
g_jingli_current_sim = collections.defaultdict(lambda :collections.defaultdict(dict))
g_jingli_following_sim = collections.defaultdict(lambda :collections.defaultdict(dict))
g_jingli_following_advance_10_sim = collections.defaultdict(lambda :collections.defaultdict(dict))

g_jingli_leading_sim_counter = 0
g_jingli_current_sim_counter = 0
g_jingli_following_sim_counter = 0
g_jingli_following_advance_10_sim_counter = 0

jingli_info = collections.defaultdict(dict)

with xlrd.open_workbook(u"经理信息.xlsx") as wb:
    ws = wb.sheet_by_index(0)
    for r in xrange(1, ws.nrows):
        funds = ws.cell(r,0).value
        half = datetime.datetime(int(ws.cell(r,1).value[:4]), 6*int(ws.cell(r,1).value[5])-5, 1)
        name = ws.cell(r,2).value
        if half not in half_date:
            print u"error at %s %s"%(funds, half)
        else:            
            jingli_info[funds][half] = name

for funds in jingli_info:
    sorted_half = sorted(jingli_info[funds].keys())
    start_idx = half_date.index(sorted_half[0])
    end_idx = half_date.index(sorted_half[-1])
    for idx in xrange(start_idx, end_idx+1):
        if half_date[idx] not in jingli_info[funds]:
            jingli_info[funds][half_date[idx]] = ["zhanwei", 1]
            
for funds in jingli_info:
    sorted_half = sorted(jingli_info[funds].keys())
    for idx, half in enumerate(sorted_half):
        if idx == 0 or jingli_info[funds][sorted_half[idx-1]][0] != jingli_info[funds][half][0]:
            if funds in g_leading_sim and half in g_leading_sim[funds]:                
                g_jingli_leading_sim_counter += 1
            if funds in g_current_sim and half in g_current_sim[funds]:                    
                g_jingli_current_sim_counter += 1
            if funds in g_following_sim and half in g_following_sim[funds]:                
                g_jingli_following_sim_counter += 1
            if funds in g_following_advance_10_sim and half in g_following_advance_10_sim[funds]:                
                g_jingli_following_advance_10_sim_counter += 1

        
        if funds in g_leading_sim and half in g_leading_sim[funds]:   
            g_jingli_leading_sim[g_jingli_leading_sim_counter][funds][half] = g_leading_sim[funds][half]
        if funds in g_current_sim and half in g_current_sim[funds]:
            g_jingli_current_sim[g_jingli_current_sim_counter][funds][half] = g_current_sim[funds][half]
        if funds in g_following_sim and half in g_following_sim[funds]:  
            g_jingli_following_sim[g_jingli_following_sim_counter][funds][half] = g_following_sim[funds][half]
        if funds in g_following_advance_10_sim and half in g_following_advance_10_sim[funds]:  
            g_jingli_following_advance_10_sim[g_jingli_following_advance_10_sim_counter][funds][half] \
                = g_following_advance_10_sim[funds][half]

In [ ]:
print jingli_info[u'000001'][datetime.datetime(2004,1,1)]
for seg_idx in g_jingli_current_sim:
    for funds in g_jingli_current_sim[seg_idx]:
        if funds == u'000001':
            sorted_quarter = sorted(g_jingli_following_sim[seg_idx][funds].keys())
            print seg_idx, funds, sorted_quarter[0], sorted_quarter[-1], len(sorted_quarter)

In [ ]:
workbook = xlsxwriter.Workbook(u'half_cosine_distance.xlsx')
current_ws = workbook.add_worksheet(u"current")
leading_ws = workbook.add_worksheet(u"leading")
following_ws = workbook.add_worksheet(u"following")
following_advance_10_ws = workbook.add_worksheet(u"following_advance_10")


def dump_result(ws, jingli_data):
    ws.write(0,0,u"fund")
    ws.write(0,1,u"quater")
    ws.write(0,2,u"distance")
    ws.write(0,3,u"num")
    ws.write(0,4,u"1-dis")
    ws.write(0,5,u"avg-4(include)")
    ws.write(0,6,u"avg-2(exclude)")
    ws.write(0,7,u"avg-4(exclude)")
    ws.write(0,8,u"avg-6(exclude)")
    
    wt_idx=1
    for seg, data in jingli_data.iteritems():
        for fund_ in data:
            sorted_quater = sorted(data[fund_].keys(), reverse=True)
            for i, quater_ in enumerate(sorted_quater):
                ws.write(wt_idx,0,fund_)
                ws.write(wt_idx,1,u"%dH%d"%(quater_.year, quater_.month/6+1))
                ws.write(wt_idx,2,data[fund_][quater_][0])
                ws.write(wt_idx,3,data[fund_][quater_][1])
                ws.write(wt_idx,4,1-data[fund_][quater_][0])

                total = 0.0
                end_idx = min((i+4, len(sorted_quater)))
                for j in xrange(i, end_idx):
                    total += (1-data[fund_][sorted_quater[j]][0])
                ws.write(wt_idx,5,total/(end_idx-i))

                if i < len(sorted_quater)-1:
                    total = 0.0
                    end_idx = min((i+3, len(sorted_quater)))
                    for j in xrange(i+1, end_idx):
                        total += (1-data[fund_][sorted_quater[j]][0])
                    ws.write(wt_idx,6,total/(end_idx-i-1))

                    total = 0.0
                    end_idx = min((i+5, len(sorted_quater)))
                    for j in xrange(i+1, end_idx):
                        total += (1-data[fund_][sorted_quater[j]][0])
                    ws.write(wt_idx,7,total/(end_idx-i-1))

                    total = 0.0
                    end_idx = min((i+7, len(sorted_quater)))
                    for j in xrange(i+1, end_idx):
                        total += (1-data[fund_][sorted_quater[j]][0])
                    ws.write(wt_idx,8,total/(end_idx-i-1))
                else:
                    ws.write(wt_idx,6,u'.')
                    ws.write(wt_idx,7,u'.')
                    ws.write(wt_idx,8,u'.')

                wt_idx += 1 
            
dump_result(current_ws, g_jingli_current_sim)
dump_result(leading_ws, g_jingli_leading_sim)
dump_result(following_ws, g_jingli_following_sim)
dump_result(following_advance_10_ws, g_jingli_following_advance_10_sim)


workbook.close()